# PRRP Experimental Framework

This notebook implements Phase 1 of the PRRP experimental framework. It supports:

1. Loading spatial (shapefile) and graph (METIS) datasets.
2. Running experiments on spatial regionalization (using PRRP from `src/spatial_prrp.py`) and graph partitioning (using PRRP from `src/graph_prrp.py` and PyMETIS from `src/pymetis_partition.py`).
3. Logging and storing performance metrics (execution time, success probability, effectiveness, and completeness).
4. Generating performance visualizations.

In [ ]:
import os
import sys
import time
import random
import logging
import json
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Add root directory to sys.path so that src modules can be imported
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT_DIR not in sys.path:
    sys.path.insert(0, ROOT_DIR)

# Set METIS DLL environment variable inside Jupyter Notebook
os.environ["METIS_DLL"] = "/opt/homebrew/Cellar/metis/5.1.0/lib/libmetis.dylib"

# Verify the variable is correctly set
print(f"METIS_DLL is set to: {os.environ.get('METIS_DLL')}")

In [ ]:
# Ensure the src directory is in the Python path
SRC_DIR = os.path.join(ROOT_DIR, "src")
if SRC_DIR not in sys.path:
	sys.path.insert(0, SRC_DIR)

In [ ]:
# Import PRRP modules
from src.prrp_data_loader import load_shapefile, load_metis_graph
from src.spatial_prrp import run_prrp, run_parallel_prrp
from src.metis_parser import load_graph_from_metis
from src.pymetis_partition import partition_graph_pymetis
from src.graph_prrp import run_graph_prrp
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import networkx as nx
from shapely.geometry import Polygon

In [ ]:
# Configure logging to display info messages
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', filename="output.txt", filemode="w")
logging.basicConfig(
    filename="output.txt",  # Log output to a file
    filemode="w",           # Overwrite the file each run
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO      # Adjust log level as needed
)
logger = logging.getLogger(__name__)

## Experiment Configuration

In this section we define:

- **Dataset paths:** For the spatial and graph datasets.
- **Parameter settings:** Such as the number of regions, sample size (M), maximum retries (MR), and number of cores (Q).
- **Output directories:** For saving results and figures.

In [ ]:
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
DATA_DIR = os.path.join(ROOT_DIR, "data")

SPATIAL_DATASET_NAME = "Tracts"  # Just a label
SPATIAL_DATASET_PATH = os.path.join(DATA_DIR, "cb_2015_42_tract_500k", "cb_2015_42_tract_500k.shp")

GRAPH_DATASET_NAME = "PGPgiantcompo"  # Just a label
GRAPH_DATASET_PATH = os.path.join(DATA_DIR, "PGPgiantcompo.graph")

In [ ]:
# Experiment parameters for spatial PRRP
P_PERCENTAGES = [0.01, 0.02, 0.03]  # p as fraction of dataset size
SPATIAL_SAMPLE_SIZES = [10, 50, 100]  # M
SPATIAL_MR_VALUES = [10, 30, 50]
SPATIAL_Q_VALUES = [1, 2, 4]

# 4) Parameter grids for graph experiments
GRAPH_P_VALUES = [5, 10, 20]   # number of partitions
GRAPH_MR_VALUES = [5, 20, 50]  # max retries
GRAPH_MS_VALUES = [5, 10]      # max partition size

In [ ]:
# Output directories for results and figures
RESULTS_DIR = os.path.join(ROOT_DIR, "results/final_results")
FIGURES_DIR = os.path.join(ROOT_DIR, "results/figures")
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

## Utility Functions for Experimentation

The following helper functions are used to:

- Load datasets (spatial or graph).
- Compute random target cardinalities for spatial regions.
- Run a spatial experiment (using PRRP).
- Run a graph partitioning experiment (using both PRRP and PyMETIS for comparison).
- Save the results in CSV/JSON format.
- Generate visualizations.

In [ ]:
def is_valid_solution(solution, cardinalities):
    """
    Checks if a solution is valid, i.e., has exactly one region
    per cardinality in 'cardinalities'.
    """
    region_sizes = sorted([len(r) for r in solution])
    sorted_cardinalities = sorted(cardinalities)
    return (region_sizes == sorted_cardinalities)

def count_valid_regions(solution, cardinalities):
    """
    Counts how many regions match any of the cardinalities.
    This helps measure partial completeness.
    """
    c_copy = list(cardinalities)
    valid_count = 0
    for region in solution:
        size_r = len(region)
        if size_r in c_copy:
            valid_count += 1
            c_copy.remove(size_r)
    return valid_count

def compute_spatial_metrics(solutions, cardinalities):
    """
    Returns {success_probability, completeness, effectiveness, num_solutions}.
    For demonstration, we set effectiveness = success_probability
    since we don't track iteration attempts here.
    """
    p = len(cardinalities)
    total_solutions = len(solutions)
    if total_solutions == 0:
        return {
            "success_probability": 0.0,
            "completeness": 0.0,
            "effectiveness": 0.0,
            "num_solutions": 0
        }
    fully_valid = 0
    sum_valid_regions = 0
    for sol in solutions:
        if is_valid_solution(sol, cardinalities):
            fully_valid += 1
        sum_valid_regions += count_valid_regions(sol, cardinalities)

    # Computing success probability
    success_probability = fully_valid / total_solutions
    
    # Computing completeness
    completeness = 0.0
    for sol in solutions:
        completeness += count_valid_regions(sol, cardinalities) / float(p)
    completeness = completeness / total_solutions

    # Computing effectiveness
    total_iterations = 1*total_solutions  # Assume 1 iteration per solution
    effectiveness = fully_valid / total_iterations

    return {
        "success_probability": success_probability,
        "completeness": completeness,
        "effectiveness": effectiveness,
        "num_solutions": total_solutions
    }

def generate_cardinalities(total_areas, num_regions):
    """
    Simple random cardinalities that sum to total_areas.
    """
    cardinalities = [1]*num_regions
    remainder = total_areas - num_regions
    for i in range(num_regions - 1):
        if remainder <= 0:
            break
        add = random.randint(0, remainder)
        cardinalities[i] += add
        remainder -= add
    
    # Add any remaining areas to the last region
    cardinalities[-1] += remainder
    
    # Shuffle the cardinalities for randomness
    random.shuffle(cardinalities)
    return cardinalities

def run_single_spatial_experiment(dataset_name, shapefile_path, p_percentage, M, MR, Q):
    """
    Runs a single experiment for the shapefile with the given parameters.
    """
    logger.info(f"Loading shapefile: {shapefile_path}")
    from src.prrp_data_loader import load_shapefile
    areas = load_shapefile(shapefile_path)
    if areas is None or len(areas) == 0:
        logger.error(f"Failed to load or empty shapefile {shapefile_path}.")
        return None

    total_areas = len(areas)
    num_regions = max(1, int(total_areas * p_percentage))
    cardinalities = generate_cardinalities(total_areas, num_regions)

    logger.info(f"Dataset={dataset_name}, total_areas={total_areas}, "
                f"p={p_percentage} -> num_regions={num_regions}, M={M}, MR={MR}, Q={Q}")
    logger.info(f"Cardinalities: {cardinalities}")

    start_t = time.time()
    solutions = run_parallel_prrp(
        areas=areas,
        num_regions=num_regions,
        cardinalities=cardinalities,
        solutions_count=M,
        num_threads=Q,
        use_multiprocessing=(Q>1)
    )
    end_t = time.time()
    exec_time = end_t - start_t

    # Compute metrics
    metrics = compute_spatial_metrics(solutions, cardinalities)

    result = {
        "dataset": dataset_name,
        "total_areas": total_areas,
        "p_percentage": p_percentage,
        "num_regions": num_regions,
        "sample_size": M,
        "MR": MR,
        "Q": Q,
        "execution_time_sec": exec_time,
        "success_probability": metrics["success_probability"],
        "completeness": metrics["completeness"],
        "effectiveness": metrics["effectiveness"],
        "num_solutions_generated": metrics["num_solutions"],
        "solutions": solutions,
    }
    return result

def run_spatial_experiments_all():
    """
    Sweeps over the parameter grids for the single shapefile,
    returns a DataFrame of results, and saves CSV/JSON.
    """
    records = []
    dataset_name = SPATIAL_DATASET_NAME
    shapefile_path = SPATIAL_DATASET_PATH

    if not os.path.isfile(shapefile_path):
        logger.error(f"Shapefile not found: {shapefile_path}")
        return pd.DataFrame()

    for pperc in P_PERCENTAGES:
        for M in SPATIAL_SAMPLE_SIZES:
            for MR in SPATIAL_MR_VALUES:
                for Q in SPATIAL_Q_VALUES:
                    logger.info(f"Running spatial experiment: p={pperc}, M={M}, MR={MR}, Q={Q}")
                    res = run_single_spatial_experiment(dataset_name, shapefile_path, pperc, M, MR, Q)
                    if res:
                        records.append(res)

    df = pd.DataFrame(records)
    df_csv = os.path.join(RESULTS_DIR, "spatial_experiments.csv")
    df_json = os.path.join(RESULTS_DIR, "spatial_experiments.json")
    df.to_csv(df_csv, index=False)
    df.to_json(df_json, orient="records", indent=2)
    logger.info(f"Saved spatial experiment results to {df_csv} and {df_json}")
    return df

## Running Experiments and Collecting Metrics

The following cells run the spatial and graph experiments over a range of parameter settings.
For demonstration, we run a single configuration for each type. In practice, you might loop over
multiple parameter combinations and aggregate the results into a DataFrame.

In [ ]:
def run_single_graph_experiment(dataset_name, graph_file_path, p, MR, MS):
    """
    Runs a single experiment for the graph dataset with given parameters,
    comparing graph-based PRRP vs. PyMETIS.
    """
    logger.info(f"Loading METIS graph from: {graph_file_path}")
    adj_list, num_nodes, num_edges = load_graph_from_metis(graph_file_path)
    if not adj_list or num_nodes==0:
        logger.error(f"Failed to load or empty graph {graph_file_path}.")
        return None

    # Run graph-based PRRP
    start_t = time.time()
    prrp_partitions = run_graph_prrp(adj_list, p, 0, MR, MS)
    prrp_time = time.time() - start_t

    # Run PyMETIS
    start_t = time.time()
    pymetis_parts = partition_graph_pymetis(adj_list, p)
    pymetis_time = time.time() - start_t

    result = {
        "graph_dataset": dataset_name,
        "num_nodes": num_nodes,
        "num_edges": num_edges,
        "p": p,
        "MR": MR,
        "MS": MS,
        "prrp_time_sec": prrp_time,
        "pymetis_time_sec": pymetis_time,
        "pymetis_parts" : pymetis_parts,
        "prrp_parts": prrp_partitions
    }
    return result

def run_graph_experiments_all():
    """
    Sweeps over the parameter grids for the single graph,
    returns a DataFrame of results, and saves CSV.
    """
    records = []
    dataset_name = GRAPH_DATASET_NAME
    graph_file_path = GRAPH_DATASET_PATH

    if not os.path.isfile(graph_file_path):
        logger.error(f"Graph file not found: {graph_file_path}")
        return pd.DataFrame()

    for pval in GRAPH_P_VALUES:
        for MR in GRAPH_MR_VALUES:
            for MS in GRAPH_MS_VALUES:
                logger.info(f"Running graph experiment: p={pval}, MR={MR}, MS={MS}")
                res = run_single_graph_experiment(dataset_name, graph_file_path, pval, MR, MS)
                if res:
                    records.append(res)

    df = pd.DataFrame(records)
    df_csv = os.path.join(RESULTS_DIR, "graph_experiments.csv")
    df.to_csv(df_csv, index=False)
    df_json = os.path.join(RESULTS_DIR, "graph_experiments.json")
    df.to_json(df_json, orient="records", indent=2)
    logger.info(f"Saved graph experiment results to {df_csv} and {df_json}")
    return df

## Generating Visualizations

Here we generate a simple visualization of the execution time from the spatial experiment.
In a full evaluation, you would plot multiple performance metrics across parameter settings.

In [ ]:
def plot_metric_vs_parameter(df, xparam, metric, title, logy=False, save_path=None):
    """
    Plots a single line: metric vs. xparam from the DataFrame.
    """
    df_sorted = df.sort_values(xparam)
    x = df_sorted[xparam]
    y = df_sorted[metric]

    plt.figure(figsize=(6, 4))
    plt.plot(x, y, marker='o', linestyle='-')
    plt.title(title)
    plt.xlabel(xparam)
    plt.ylabel(metric)
    if logy:
        plt.yscale("log")
    plt.grid(True)
    if save_path:
        plt.savefig(save_path, dpi=300)
        logger.info(f"Saved figure to {save_path}")
    plt.show()

In [ ]:
# Visualizing the regions in shapefile
def visualize_shapefile(shapefile_path, regions):
    """
    Visualizes the shapefile with regions highlighted.
    """
    logger.info(f"Visualizing shapefile: {shapefile_path}")
    gdf = gpd.read_file(shapefile_path)
    gdf["region"] = -1  # Default region
    for i, region in enumerate(regions):
        gdf.loc[gdf["GEOID"].isin(region), "region"] = i
    gdf.plot(column="region", cmap="tab20", figsize=(10, 10))
    plt.title("Regions in Shapefile")
    plt.show()

# Example: visualize the first solution from the spatial experiments
if not df_spatial.empty:
    visualize_shapefile(SPATIAL_DATASET_PATH, df_spatial.iloc[0]["solutions"][0])

# Visualizing the graph partitions
def visualize_graph_partitions(graph_file_path, partitions):
    """
    Visualizes the graph partitions.
    """
    logger.info(f"Visualizing graph partitions: {graph_file_path}")
    G = load_metis_graph(graph_file_path)
    pos = nx.spring_layout(G)
    colors = np.linspace(0, 1, len(partitions))
    for i, partition in enumerate(partitions):
        nx.draw_networkx_nodes(G, pos, nodelist=partition, node_size=50, node_color=[colors[i]]*len(partition))
    nx.draw_networkx_edges(G, pos, alpha=0.5)
    plt.title("Graph Partitions")
    plt.show()

# Example: visualize the first solution from the graph experiments
if not df_graph.empty:
    visualize_graph_partitions(GRAPH_DATASET_PATH, df_graph.iloc[0]["prrp_parts"])

## Experiment Suite Execution 

Finally we setup and run the experimentation for our spatial and graph datasets and plot the results

In [ ]:
if __name__ == "__main__":
    logger.info("=== Starting Spatial Experiments ===")
    df_spatial = run_spatial_experiments_all()
    logger.info("=== Spatial Experiments Completed ===")

    # Example: let's pick a single slice of df_spatial to plot execution time vs. M
    # Suppose we fix p_percentage=0.01, MR=10, Q=1
    subdf = df_spatial[
        (df_spatial["p_percentage"] == 0.01) &
        (df_spatial["MR"] == 10) &
        (df_spatial["Q"] == 1)
    ]
    # We can plot "execution_time_sec" vs. "sample_size"
    if not subdf.empty:
        plot_metric_vs_parameter(
            subdf,
            xparam="sample_size",
            metric="execution_time_sec",
            title="Execution Time vs. Sample Size (p=1%, MR=10, Q=1)",
            logy=True,
            save_path=os.path.join(FIGURES_DIR, "figure_spatial_time_vs_M.png")
        )

    logger.info("=== Starting Graph Experiments ===")
    df_graph = run_graph_experiments_all()
    logger.info("=== Graph Experiments Completed ===")

    # Example: plot PRRP time vs. p
    # Suppose we fix MR=5, MS=5
    subdf_g = df_graph[(df_graph["MR"] == 5) & (df_graph["MS"] == 5)]
    if not subdf_g.empty:
        plot_metric_vs_parameter(
            subdf_g,
            xparam="p",
            metric="prrp_time_sec",
            title="Graph PRRP Time vs. p (MR=5, MS=5)",
            logy=True,
            save_path=os.path.join(FIGURES_DIR, "figure_graph_time_vs_p.png")
        )

    logger.info("All experiments completed. Results and figures are saved.")